In [138]:
import pandas as pd



In [139]:
df1 = pd.read_excel("CO2_emissions/Table 11.01.xlsx", skiprows=8)
df1 = df1[["Month", "Total Energy CO2 Emissions"]]


df2 = pd.read_excel("CO2_emissions/Table 11.02.xlsx", skiprows=8)
df2 = df2[["Month", "Total Energy Residential Sector CO2 Emissions"]]


df3 = pd.read_excel("CO2_emissions/Table 11.03.xlsx", skiprows=8)
df3 = df3[["Month", "Total Energy Commercial Sector CO2 Emissions"]]

df4 = pd.read_excel("CO2_emissions/Table 11.04.xlsx", skiprows=8)
df4 = df4[["Month", "Total Industrial Sector CO2 Emissions"]]


df5 = pd.read_excel("CO2_emissions/Table 11.05.xlsx", skiprows=8)
df5 = df5[["Month", "Total Energy Transportation Sector CO2 Emissions"]]

df6 = pd.read_excel("CO2_emissions/Table 11.06.xlsx", skiprows=8)
df6 = df6[["Month", "Total Energy Electric Power Sector CO2 Emissions"]]


df7 = pd.read_excel("CO2_emissions/Table 11.07.xlsx", skiprows=8)
df7 = df7[["Month", "Biomass CO2 Emissions"]]


In [140]:
dfs = [df1, df2, df3, df4, df5, df6, df7]

df_merged = dfs[0]
for df in dfs[1:]:
    df_merged = df_merged.merge(df, on='Month', how='inner')


In [141]:
cols_to_sum = [
    "Total Energy CO2 Emissions",
    "Total Energy Residential Sector CO2 Emissions",
    "Total Energy Commercial Sector CO2 Emissions",
    "Total Industrial Sector CO2 Emissions",
    "Total Energy Transportation Sector CO2 Emissions",
    "Total Energy Electric Power Sector CO2 Emissions",
    "Biomass CO2 Emissions"
]

df_merged["Total_CO2_Emissions"] = df_merged[cols_to_sum].sum(axis=1)

In [142]:
# df_merged.to_csv("merged_data.csv", index = False)

In [143]:
df_new  = df_merged[["Month","Total_CO2_Emissions"]]

In [144]:
df_new = df_new.drop(index=0).reset_index(drop=True)

In [145]:
df_new.head(5)

,Month,Total_CO2_Emissions
0,1973 January,1031.918
1,1973 February,940.42
2,1973 March,926.209
3,1973 April,841.536
4,1973 May,862.351


In [146]:
df_new[['Year', 'Month_Name']] = df_new['Month'].str.extract(r'(\d{4})\s+(\w+)', expand=True)

month_map = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04',
    'May': '05', 'June': '06', 'July': '07', 'August': '08',
    'September': '09', 'October': '10', 'November': '11', 'December': '12'
}
df_new['Month'] = df_new['Month_Name'].map(month_map)
df_new = df_new[["Year", "Month","Total_CO2_Emissions"]]

In [147]:
df_new.head(5)

,Year,Month,Total_CO2_Emissions
0,1973,01,1031.918
1,1973,02,940.42
2,1973,03,926.209
3,1973,04,841.536
4,1973,05,862.351


In [148]:
df_transportation = pd.read_csv("Transportation_petroleum_consumption/MER_T03_07C.csv")
df_transportation = df_transportation[df_transportation["Description"] == "Aviation Gasoline Consumed by the Transportation Sector"]
df_transportation = df_transportation[["YYYYMM","Value"]]


In [149]:
df_transportation.head(5)

,YYYYMM,Value
0,194913,93.129
1,195013,108.266
2,195113,145.071
3,195213,169.391
4,195313,193.732


In [150]:
df_transportation['Year'] = df_transportation['YYYYMM'].astype(str).str[:4]
df_transportation['Month'] = df_transportation['YYYYMM'].astype(str).str[4:]

df_transportation = df_transportation[df_transportation['Month'] != '13'].reset_index(drop=True)
df_transportation = df_transportation[["Year","Month","Value"]]
df_transportation.rename(columns = {"Value" :"Petroleum Consumption"}, inplace= True)


In [151]:
df_transportation.head(5)

,Year,Month,Petroleum Consumption
0,1973,01,39.516
1,1973,02,44.143
2,1973,03,42.419
3,1973,04,42.567
4,1973,05,50.71


In [152]:
df_new  = df_new.merge(df_transportation, how = "inner", on = ["Year","Month"])

In [153]:
df_new.head(5)

,Year,Month,Total_CO2_Emissions,Petroleum Consumption
0,1973,01,1031.918,39.516
1,1973,02,940.42,44.143
2,1973,03,926.209,42.419
3,1973,04,841.536,42.567
4,1973,05,862.351,50.71
